📖 참고 : 딥 러닝을 이용한 자연어 처리 입문

# Doc2Vec과 추천 시스템을 이용한 대학 학과 추천_ver1.0

- 캡스톤종합디자인프로젝트 논문 후보로 해당 미니 프로젝트를 진행함
- 대입 서류를 기반으로 대학 학과를 추천 받음으로써 아직 학과를 선택하지 못 한 학생들에게 도움을 줄 수 있음
- 또한 현재 대한민국에 존재하는 대학과 학과의 수가 매우 많기 때문에 학과 선택의 폭을 넓혀줄 수 있음
- 데이터 수집 : [Doc2Vec과 추천 시스템을 이용한 대학 학과 추천_데이터 수집.ipynb](https://github.com/ahyeon0508/DS_NLP/blob/master/Doc2Vec/Doc2Vec%EA%B3%BC%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C%EC%9D%84%20%EC%9D%B4%EC%9A%A9%ED%95%9C%20%EB%8C%80%ED%95%99%20%ED%95%99%EA%B3%BC%20%EC%B6%94%EC%B2%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%88%98%EC%A7%91.ipynb)
- ver1.0은 대입 서류를 기반으로 대학 학과를 추천x

---

# 1. 라이브러리 호출 및 데이터 불러오기

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("career_major.csv")

In [3]:
data.head()

,Unnamed: 0,학과개요,흥미와적성,학과특성,대학
0,0,"가정교육과의 교육과정은 크게 아동학, 가정관리학, 의류학, 식품학, 영양학, 주거학...","평소 친구, 가족, 주변 사람 등 사람에 대해 관심이 많거나 주변 사람의 고민을 들...","영양학, 의류학, 주거학, 가족학 등의 교과내용학을 배웁니다. 가정과교육론, 가정과...","['가정교육과(고려대학교) ', '가정교육과(동국대학교) ', '가정교육과(중앙대학..."
1,1,가족복지과는 사회복지에 대한 새로운 관점에 맞게 가족의 기능을 강화하고 잠재력을 개...,"가족복지학을 공부하기 위해서는 평소 어려운 사람을 돕는 일을 좋아하고, 심리적ㆍ정서...",저출산 고령화 사회가 되면서 가족의 기능 강화 및 돌봄 서비스에 대한 사회적 요구가...,"['가족복지과(자연)(숭의여자대학교) ', '가족복지과(숭의여자대학교) ', '사회..."
2,2,간호학은 환자의 건강을 유지ㆍ증진시키고 질병의 고통을 줄여 행복한 삶을 살 수 있도...,"인체나 질병, 생명 등에 대한 관심이 있고, 남을 도와주는 봉사정신이 있는 사람이면...",아픈 사람을 돌보며 다른 사람을 도우며 사는 삶에 대해 배울 수 있는 곳이 간호과입...,"['간호학과(삼육보건대학교) ', '간호학과(4년제)(서울여자간호대학교) ', '간..."
3,3,나이팅게일 선서에 대해 들어본 적 있나요? 간호사의 역할과 마음가짐에 대한 선서입니...,"인체나 질병, 생명 등에 대한 관심이 있고, 남을 도와주는 것을 좋아하는 사람이면 ...",신체적으로나 정신적으로 건강하게 일생을 살아가는 것은 가장 큰 복이라고 할 수 있습...,"['간호학과(가톨릭대학교) ', '간호학과(경희대학교) ', '간호학과(고려대학교)..."
4,4,"우리가 살아가는 데에 많은 것들이 필요하지만, 가장 먼저 몸과 정신이 건강해야합니다...",평소 식단관리나 생활운동 등 건강에 대하여 관심이 많으면 좋습니다. 사람들을 대상으...,오늘날 인간의 수명은 점차 증가하고 있습니다. 그러나 현대인들은 운동량 부족과 불균...,"['헬스케어매니지먼트과(동의과학대학교) ', '스포츠ㆍ건강관리과(부산경상대학교) '..."


In [4]:
data.drop(data.columns[0], axis=1, inplace=True)

# 2. EDA 및 전처리

In [5]:
len(data)

502

## 학과정보 데이터 생성

학과개요, 흥미와적성, 학과특성을 합친 데이터임

In [6]:
data["학과정보"] = data["학과개요"] + data["흥미와적성"] + data["학과특성"]

In [7]:
data.head()

,학과개요,흥미와적성,학과특성,대학,학과정보
0,"가정교육과의 교육과정은 크게 아동학, 가정관리학, 의류학, 식품학, 영양학, 주거학...","평소 친구, 가족, 주변 사람 등 사람에 대해 관심이 많거나 주변 사람의 고민을 들...","영양학, 의류학, 주거학, 가족학 등의 교과내용학을 배웁니다. 가정과교육론, 가정과...","['가정교육과(고려대학교) ', '가정교육과(동국대학교) ', '가정교육과(중앙대학...","가정교육과의 교육과정은 크게 아동학, 가정관리학, 의류학, 식품학, 영양학, 주거학..."
1,가족복지과는 사회복지에 대한 새로운 관점에 맞게 가족의 기능을 강화하고 잠재력을 개...,"가족복지학을 공부하기 위해서는 평소 어려운 사람을 돕는 일을 좋아하고, 심리적ㆍ정서...",저출산 고령화 사회가 되면서 가족의 기능 강화 및 돌봄 서비스에 대한 사회적 요구가...,"['가족복지과(자연)(숭의여자대학교) ', '가족복지과(숭의여자대학교) ', '사회...",가족복지과는 사회복지에 대한 새로운 관점에 맞게 가족의 기능을 강화하고 잠재력을 개...
2,간호학은 환자의 건강을 유지ㆍ증진시키고 질병의 고통을 줄여 행복한 삶을 살 수 있도...,"인체나 질병, 생명 등에 대한 관심이 있고, 남을 도와주는 봉사정신이 있는 사람이면...",아픈 사람을 돌보며 다른 사람을 도우며 사는 삶에 대해 배울 수 있는 곳이 간호과입...,"['간호학과(삼육보건대학교) ', '간호학과(4년제)(서울여자간호대학교) ', '간...",간호학은 환자의 건강을 유지ㆍ증진시키고 질병의 고통을 줄여 행복한 삶을 살 수 있도...
3,나이팅게일 선서에 대해 들어본 적 있나요? 간호사의 역할과 마음가짐에 대한 선서입니...,"인체나 질병, 생명 등에 대한 관심이 있고, 남을 도와주는 것을 좋아하는 사람이면 ...",신체적으로나 정신적으로 건강하게 일생을 살아가는 것은 가장 큰 복이라고 할 수 있습...,"['간호학과(가톨릭대학교) ', '간호학과(경희대학교) ', '간호학과(고려대학교)...",나이팅게일 선서에 대해 들어본 적 있나요? 간호사의 역할과 마음가짐에 대한 선서입니...
4,"우리가 살아가는 데에 많은 것들이 필요하지만, 가장 먼저 몸과 정신이 건강해야합니다...",평소 식단관리나 생활운동 등 건강에 대하여 관심이 많으면 좋습니다. 사람들을 대상으...,오늘날 인간의 수명은 점차 증가하고 있습니다. 그러나 현대인들은 운동량 부족과 불균...,"['헬스케어매니지먼트과(동의과학대학교) ', '스포츠ㆍ건강관리과(부산경상대학교) '...","우리가 살아가는 데에 많은 것들이 필요하지만, 가장 먼저 몸과 정신이 건강해야합니다..."


## 정규 표현식을 통한 한글 외 문자 제거

In [8]:
data["학과정보"] = data['학과정보'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

## 불용어 정의

In [9]:
stopwords = ['은', '는', '이', '가', '을', '를', '합니다', '입니다', '등', '의', '에', '대해', '과']

## 토큰화 작업 및 불용어 제거

In [10]:
okt = Okt()
tokenized_data = []
for sentence in data["학과정보"]:
    token = okt.morphs(sentence, stem=True) # 토큰화
    token = [word for word in token if not word in stopwords] # 불용어 제거
    token = " ".join(token)
    tokenized_data.append(token)

In [11]:
data["학과정보"] = tokenized_data

# 3. Word2Vec 모델 생성

In [12]:
word2vec_model = Word2Vec(sentences = data["학과정보"], size = 100, window = 5, min_count = 2, workers = 4, sg = 1)

# 4. Doc2Vec

## 단어 벡터의 평균 구하기

In [13]:
def vectors(major_info):
    document_embedding_list = []

    # 각 학과에 대해서
    for words in major_info:
        doc2vec = None
        count = 0
        for word in words:
            if word in word2vec_model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [14]:
document_embedding_list = vectors(data["학과정보"])
print('문서 벡터의 수 :',len(document_embedding_list))

문서 벡터의 수 : 502


# 5. 추천 시스템 구현하기

## 코사인 유사도

In [15]:
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

코사인 유사도 매트릭스의 크기 : (502, 502)


In [16]:
def recommendations(idx):
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]
    
    major_indices = [i[0] for i in sim_scores]
    
    return data.iloc[major_indices]["대학"]

## 가족복지과와 비슷한 학과

In [17]:
idx = 1

recommendations(idx)

190    ['사회복지학과(명지전문대학) ', '사회복지과(명지전문대학) ', '사회복지학부(...
192    ['사회복지학부(경희사이버대학교) ', '사회복지전공(경희사이버대학교) ', '평생...
191    ['사회서비스상담복지과(명지전문대학) ', '사회복지상담과(부산여자대학교) ', '...
85     ['노인케어창업과(삼육보건대학교) ', '노인복지과(삼육보건대학교) ', '노인헬스...
333    ['일반사회교육과(부산대학교) ', '사회교육학부 일반사회교육전공(경북대학교) ',...
Name: 대학, dtype: object

## 간호과와 비슷한 학과

In [18]:
idx = 2

recommendations(idx)

3      ['간호학과(가톨릭대학교) ', '간호학과(경희대학교) ', '간호학과(고려대학교)...
313    ['윤리문화학전공(동국대학교) ', '윤리문화학과(동국대학교) ', '윤리문화학과(...
312    ['윤리교육과(서울대학교) ', '윤리교육과(성신여자대학교) ', '윤리교육과(경성...
390    ['공간정보공학과(서울시립대학교) ', '공간정보공학과(세종대학교) ', '지구정보...
33     ['뇌교육융합학과(글로벌사이버대학교) ', '과학교육계열(서울대학교) ', '과학교...
Name: 대학, dtype: object

## 건축공학과와 비슷한 학과

In [19]:
idx = 8

recommendations(idx)

11     ['건축학부(건국대학교) ', '건축설계전공(건국대학교) ', '주거환경전공(건국대...
436    ['사회환경플랜트공학과(건국대학교) ', '토목공학과(건국대학교) ', '토목 ㆍ ...
437    ['토목공학과(명지전문대학) ', '토목과(명지전문대학) ', '토목공학과(서일대학...
10     ['건설계열(부산과학기술대학교) ', '건설공병학과(대덕대학교) ', '친환경건축설...
378    ['크루즈해양ㆍ인테리어과(동주대학교) ', '조선해양과(동주대학교) ', '조선해양...
Name: 대학, dtype: object

----

# 결론 및 향후 방향

- 학생의 대입 서류와 학과정보의 코사인 유사도를 통해 학과 추천해주는 방법 마련하기
- 데이터 구축 다시 하기
    - 학과명, 관련직업, 공부하는 주요 교과목 등 넣기
- 데이터가 502개로 매우 부족함 => 방법 찾기
- 추천 정확도 높이기